<a href="https://colab.research.google.com/github/palit-ishan/Data-Pipeline/blob/main/Data_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>